![](https://raw.githubusercontent.com/unm-escape/escape2022/main/header2.png)

<h1 style="text-align:center; color:#3333ff;">Intro to ObsPy</h1>
<br>
<div style="text-align:center; font-size:16px">
    Earth and Planetary Sciences,<br>
    University of New Mexico<br>
    <br>
    August 10, 2023
</div>

# [ObsPy](https://docs.obspy.org/)

ObsPy is an open-source project dedicated to provide a Python framework for processing seismological data. It provides parsers for common file formats, clients to access data centers and seismological signal processing routines which allow the manipulation of seismological time series.

---

**ObsPy Tutorial Outline:**
1. Downloading station inventory
2. Downloading earthquake catalog
3. Downloading teleseismic earthquake seismograms<br>
4. Downloading local earthquake seismograms
5. Filtering waveforms


---
**Dependencies:** Obspy, Matplotlib, Numpy

---

<pre>




</pre>


## How to install ObsPy: https://anaconda.org/conda-forge/obspy

Enter this is your Anaconda Terminal: **conda install -c conda-forge obspy**

![](https://i.ytimg.com/vi/kegL0pBpHsM/maxresdefault.jpg)



## Seismologists often work with 3 kinds of data
### We're going to learn how to download all of these kinds of data

<br>

![](http://www.seismo.ethz.ch/export/sites/sedsite/research-and-teaching/.galleries/img_products_software/WaveformData1.jpg_137093282.jpg)
### (1) Waveform data - The actual data recorded by seismometers around the world 
These waveforms are maintained by a global group of seismic networks that upload their data to the web

<pre>




</pre>

![](http://eqseis.geosc.psu.edu/cammon/HTML/Classes/AdvSeismo/SI/images/docs/getting_information_on_seismic_stations/5b2f20bf-823a-4845-a574-a69773e96e6f.png)
### (2) Meta-data - Data that tell us about the seismometer
Where it is, how many channels it has, its mechanical system properties, how quickly it samples, etc.

<pre>




</pre>




![](https://static.temblor.net/wp-content/uploads/2022/12/gorda-usgs-1964-M6pt35.jpg)
### (3) Earthquake Catalogs - the USGS maintains US earthquake catalogs ([ANSS Comprehensive Catalog](https://www.usgs.gov/tools/search-earthquake-catalog))
This is a giant table that contains standardized information for recorded earthquakes from across the world. 



<pre>




</pre><pre>




</pre><pre>




</pre>
## 1. Downloading station inventory

ObsPy can retrieve seismic data from many different online databases globally. These databases, known as "clients" in the ObsPy ecosystem are coordinated by the International Federation of Digital Seismograph Networks ([FDSN](https://docs.obspy.org/packages/obspy.clients.fdsn.html)).

In order to retrieve station metadata, earthquake catalogs or seismograms with ObsPy, a client object must be specified. In this tutorial we will use the IRIS webservice.

More information on basic FDSN client usage can be found [here](https://docs.obspy.org/packages/obspy.clients.fdsn.html#basic-fdsn-client-usage).

---

<pre>




</pre><pre>




</pre>Let's begin by importing fdsn client package.

In [ ]:
from obspy.clients.fdsn import Client as FDSN_Client

Now we specify that we want data from the IRIS web service.

In [ ]:
client = FDSN_Client("IRIS")


<pre>




</pre><pre>




</pre>
## In order to download data, we need to know:
### (1) The time interval
### (2) The location of the seismometer or earthquake
### (3) The type of seismometer you want to use

<pre>




</pre>
## TIME

**ObsPy uses the [UTCDateTime](https://docs.obspy.org/packages/autogen/obspy.core.utcdatetime.UTCDateTime.html) class to work with time. So we have to import the UTCDatetime package too.**

UTCDateTime objects work very similarly to datetime object, but they have **nanosecond** precision and always corresponds to UTC time unless specified otherwise.



In [ ]:
from obspy import UTCDateTime

<pre>




</pre>
**UTCDateTime model can handle string representations of time**

In [ ]:
UTCDateTime('2023-08-10')

<pre>




</pre>
**It can also take the number of seconds since the beginning of UTCDateTime**

In [ ]:
UTCDateTime(120)


<pre>




</pre>
**Let's say you know the [Tohoku, Japan](https://en.wikipedia.org/wiki/2011_T%C5%8Dhoku_earthquake_and_tsunami) earthquake occured on March 11, 2011 at 5:46:24 UTC and you eventually want to request seismic data for an entire day after that moment.** 

Use the UTCDateTime object to create variables that contain the start time of the earthquake and exactly one day afterwards.

In [ ]:
start = UTCDateTime(year = 2011, month = 3, day = 11, hour  = 5, minute = 36, second = 24)
print(start)
end = start + 60*60*24 
print(end)

<pre>




</pre><pre>




</pre>
### LOCATION

**How do we decide which stations to download?**

Generally, seismologists use the [SEED](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed/) (Standard for the Exchange of Earthquake Data) file convention.

**SEED data include** 
1) A data stream - this is a data timeseries
2) Metadata that provide all the info about the instrument


**To request SEED data**, we need at least 4 components to specify the seismometer we'd like to interface with:
1) Network code
2) Station code
3) Location ID
4) Channel codes

We can check global seismic station distribution and get the identification information from the IRIS [GMAP](https://ds.iris.edu/gmap/) tool which creates dynamic station maps that can be panned and zoomed. In this example, I retrieved network code, station code, channel codes and the location ID from the GMAP tool. The starttime and endtime depend on when the events occured.

<pre>




</pre>
### We can declare the station location info and the start/end time

In [ ]:
network = 'IU'
station = 'ANMO'
channels = 'LH?'
location = '00'



<pre>




</pre>
### Then we use the *get_stations()* method to grab the metadata of a known station

In [ ]:
inventory_single = client.get_stations(network=network, station=station, channel=channels, 
                                level='response', location=location, starttime=start, 
                                endtime=end)


<pre>




</pre>
### The inventory object contains the station metadata

In [ ]:
vars(inventory_single[0])


<pre>




</pre>
### Or we can tell ObsPy that we want to get all the stations within a certain radius.

In [ ]:
lat = 35
lon = -106.65
radius = 5 ## This is 1 arc degree of Earth's surface ~111.32 km

inventory_many = client.get_stations(level='response', location=location, starttime=start, 
                                endtime=end, latitude = lat, longitude = lon, maxradius = radius)

<pre>




</pre>
### How many networks and stations were in operation near Albuquerque during the Tohoku Earthquake?

In [ ]:
print(inventory_many)


<pre>




</pre>
### TYPE of instrument

#### How can we decide which channels to use?

[IRIS](https://ds.iris.edu/ds/nodes/dmc/data/formats/seed-channel-naming/) has standarized channel names that correspond to different types of seismometers and their orientation.

#### This convention has 3 important codes 
(1) The Band Code corresponds to the frequency sensitivity <br>
(2) The Instrument Code tells us what type of instrument <br>
(3) The Orientation Code tells us the vertical and horizontal orientation

<pre>




</pre>
### We can quickly make a plot of our instruments' locations

In [ ]:
map = inventory_many.plot('local')


<pre>




</pre>
<pre>




</pre>

## 2. Downloading earthquake catalog

For this example, let's create a catalog for the 2011 [M9.0 Tohoku earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/ak0219neiszm/executive).

We will begin by querying the events service of the FDSN client using the [get_events](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_events.html) function which returns an events catalog object.

In [ ]:
client = FDSN_Client("IRIS")
catalog = client.get_events(starttime=start, endtime=end, 
                            minmagnitude=0, latitude = 38.3, 
                            longitude=142.4,maxradius=1)

In [ ]:
print(catalog)

<pre>




</pre>

### ObsPy has built-in tools to help us quickly visualize the earthquakes

In [ ]:
catalog.plot(projection='local', label=None);

<pre>




</pre>

### How can we grab info pertaining to individual earthquakes in the catalog?

Let's take the length of the catalog object

In [ ]:
print(len(catalog))

<pre>




</pre>
### This tells us that there is a value we can index for every earthquake in the catalog.

Let's grab the first earthquake.

In [ ]:
eq1 = catalog[0]
print(type(eq1))

<pre>




</pre>

### This is an obspy [event](https://docs.obspy.org/packages/autogen/obspy.core.event.event.Event.html) object.  We can grab all kinds of information about the earthquake from this event object.

In [ ]:
print('The event is an',eq1.event_type)
print('Its magnitude was', eq1.magnitudes[0].mag)
print('It happened at', eq1.origins[0].depth/1000, 'kilometers below sea level')

To read event(s) from file, import ObsPy's [read_events()](https://docs.obspy.org/master/packages/autogen/obspy.core.event.read_events.html) function.

<pre>




</pre><pre>




</pre>

## 3. Downloading earthquake seismograms

Lets download seismograms for the 2011 [Tohoku Earthquake](https://earthquake.usgs.gov/earthquakes/eventpage/official20110311054624120_30/origin/phase?source=official&code=official20110311054624120_30) recorded at your chosen station.



<pre>




</pre>
### First we define our Client

In [ ]:
from obspy.clients.fdsn import Client as FDSN_Client
from obspy import UTCDateTime

client = FDSN_Client("IRIS")


<pre>




</pre>
### Next we need our station location and time interval.

In [ ]:
sta_lat = inventory_single[0].stations[0].latitude
sta_lon = inventory_single[0].stations[0].longitude
network = inventory_single[0].code
station = inventory_single[0].stations[0].code
location = '00' 
channel = 'BH?'

tstart = start
tend = end

<pre>




</pre>
### We request waveforms with the [client.get_waveforms()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html) method. This needs the station info and time interval.

In [ ]:
st = client.get_waveforms(network, station, location, channel, tstart, tend)
print(st)

<pre>




</pre>
### The output data type is called a [***stream***](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html), which contains traces of data for every channel and metadata that tells us how the instrument recorded the data.

<br>

### Let's plot a whole day of seismic data! We can do this with the stream object's plot() method

In [ ]:
whole_day = st.plot()

<pre>




</pre>
### Let's window the data so we only see 3 hours of data after the event origin time.

In [ ]:
short = st.copy()
short = short.trim(endtime = start + 3*60*60)
short.plot().show

<pre>




</pre>

### Because our recording starts at the approximate origin time, we know it took ~22 minutes for the first energy of the Tohoku earthquake to reach Albuquerque. 
<br>

#### What units are we looking at? Raw seismic data are downloaded as [***counts***](https://ds.iris.edu/ds/support/faq/6/what-is-a-count-in-timeseries-data/). We need to use a build-in mathematical function to convert ***counts*** to displacement, velocity, or acceleration. 



<pre>




</pre><pre>




</pre>
## Seismic waves contain a combination of many different frequency waves. 

### We can visualize this with a plot called a [***spectrogram***](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.spectrogram.html)

In [ ]:
spec = st[0].spectrogram()

In [ ]:
spec = st[0].spectrogram(dbscale=True)


<pre>




</pre><pre>




</pre>
## Now, let's grab a local earthquake in Albuquerque on the same station. 

In [ ]:
client = FDSN_Client("IRIS")
start = UTCDateTime(year=2023,month = 1,day=1)
end = UTCDateTime.now()

catalog = client.get_events(starttime=start, endtime=end, 
                            minmagnitude=0, latitude = sta_lat, 
                            longitude=sta_lon,maxradius=1)

In [ ]:
print(catalog)


fig = inventory_single.plot(show=False, projection='local')  

plt = catalog.plot(fig=fig, projection='local')  


<pre>




</pre>
### Imagine we want to grab data for ~2 minutes after the first earthquake occurs. <br>

We need to know when the earthquake occured. 



In [ ]:
teq = catalog[0].origins[0].time
teq_end = teq + 2*60


In [ ]:
st = client.get_waveforms(network, station, location, channel, teq, teq_end)

In [ ]:
local_event = st.plot()

<pre>




</pre>
### Take the spectrogram of this event

In [ ]:
spec = st.spectrogram()

<pre>




</pre>
### We can remove waves of certain frequencies by [***filtering***](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.filter.html). 

#### Common kinds of filters are low-pass, band-pass, and high-pass. Let's do a high pass and low pass of 0.25 Hz and see what we get. 



In [ ]:
low = st[2].copy().filter('lowpass', freq = 1)
high = st[2].copy().filter('highpass', freq = 1)
low_fig = low.plot()
high_fig = high.plot()

<pre>




</pre>
### What if we don't  have a good idea of how long it will take for the earthquake to arrive at our station?

#### We can use something called a [Tau-P model](https://docs.obspy.org/packages/obspy.taup.html). This is a prediction for how long it should take for certain earthquake phases to arrive with a very simple Earth velocity. 

#### To do this, we only need to know the distance between the earthquake and the station, the earthquake's depth, and the phase.

In [ ]:
## import modules for calculating TauPy and distance on the Earth's surface.
from obspy.taup import TauPyModel
from obspy.geodetics import gps2dist_azimuth, kilometer2degrees

## Define Earth's velocity model for TauP
tpmodel = TauPyModel(model='iasp91')

<pre>




</pre>
### We can grab an earthquake's position from the catalog

In [ ]:
## First, grab the earthquake's position.
eq_lat = catalog[0].origins[0].latitude
eq_lon = catalog[0].origins[0].longitude
eq_dep = catalog[0].origins[0].depth/1000


In [ ]:
### Now, use the distance functions and Tau-P model to estimate the P-wave arrival time.
epi_dist_m, az, baz = gps2dist_azimuth(eq_lat, eq_lon, sta_lat, sta_lon)

epi_dist_km = epi_dist_m/1000
epi_dist_deg = kilometer2degrees(epi_dist_km)
arrivals = tpmodel.get_travel_times(distance_in_degree=epi_dist_deg, source_depth_in_km=eq_dep, phase_list=['P'])
print(arrivals[0].time)

<pre>




</pre>
## Let's see how close the P-onset appears to be compared to this modeled arrival. 

In [ ]:

fig = st[2].plot(show=False, handle=True)

import matplotlib.pyplot as plt

ax = fig.axes[0]
ax.axvline(teq+arrivals[0].time, ymin = -40000, ymax = 40000)
plt.show()

<pre>




</pre><pre>




</pre>
## You have two major tasks.
### (1) Plot a global map of all earthquakes that occured on your birthday.  
Grab the largest earthquake that occured on your brithday. <br> Download the data on a station of your choice. <br> If you can't find the event, you might try a different station, closer to the epicenter. <br> Plot the event and its spectrogram. <br> Explore different channels and filtering (remember to copy the stream before filtering). 
### (2) Plot the seismic network and earthquakes near your place of birth. 
Grab an inventory of all seismometers in operation since 1970. Plot it. <br>
Grab a catalog of all earthquakes in this area since 1970. Plot it. <br>
Grab the largest earthquake near your hometown since 1970. Download the waveform data for a station of your choice. Plot and filter the data for this largest earthquake.

<pre>




</pre><pre>




</pre>

## There are tons of things you can do with ObsPy for an final project. Just let us know what interests you can we can guide you to the right tools. 
### You could explore earthquake statistics in earthquake catalogs
![](https://www.researchgate.net/publication/281368240/figure/fig3/AS:279767048835082@1443713070722/Cumulative-seismic-moment-release-and-number-of-earthquakes-by-day-modified-from-Pedrera.png)
<pre>




</pre>

### You could estimate the local magnitude of a couple events and compare with their catalog magnitudes. 
![](https://opentextbc.ca/physicalgeology2ed/wp-content/uploads/sites/298/2019/06/P-and-S-waves.png)
<pre>




</pre>

### You could look at seasonal variations in seismic background noise using long-term spectrograms. 
![](https://www.seismosoc.org/Publications/BSSA_html/bssa_104-6/2014079-esupp/2014079_esupp_Figure_S3.png)